In [1]:
#external imports
import sys
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import copy
import math

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_precision = torch.float32
model_precision_str = "float32"
serial_num_gpu = 0
sample_size = 1000
layers = [1]

In [3]:
#setup and internal imports
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.insert(0,parent_directory)
model_name = 'lmsys/vicuna-7b-v1.5' #'mistralai/Mistral-7B-instruct-v0.2'
from rome.layer_stats import layer_stats

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype = model_precision)

tokenizer_config.json: 100%|██████████| 749/749 [00:00<00:00, 4.20MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 10.5MB/s]
special_tokens_map.json: 100%|██████████| 438/438 [00:00<00:00, 2.98MB/s]


In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = model_precision)
model.cuda(serial_num_gpu)

config.json: 100%|██████████| 615/615 [00:00<00:00, 3.82MB/s]
pytorch_model.bin.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 57.4MB/s]
pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]
pytorch_model-00001-of-00002.bin:   0%|          | 21.0M/9.98G [00:00<00:49, 203MB/s]
pytorch_model-00001-of-00002.bin:   1%|          | 52.4M/9.98G [00:00<00:44, 224MB/s]
pytorch_model-00001-of-00002.bin:   1%|          | 83.9M/9.98G [00:00<00:42, 233MB/s]
pytorch_model-00001-of-00002.bin:   1%|          | 115M/9.98G [00:00<00:42, 233MB/s] 
pytorch_model-00001-of-00002.bin:   1%|▏         | 147M/9.98G [00:00<00:41, 236MB/s]
pytorch_model-00001-of-00002.bin:   2%|▏         | 178M/9.98G [00:00<00:41, 238MB/s]
pytorch_model-00001-of-00002.bin:   2%|▏         | 210M/9.98G [00:00<00:40, 239MB/s]
pytorch_model-00001-of-00002.bin:   2%|▏         | 241M/9.98G [00:01<00:40, 239MB/s]
pytorch_model-00001-of-00002.bin:   3%|▎         | 273M/9.98G [00:01<00:40, 238MB/s]
pytorch_

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 

In [6]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [7]:
model_name = model.config._name_or_path.replace("/", "_")

In [8]:
for layer in layers:
    stat = (layer_stats(
        model = model,
        tokenizer = tokenizer,
        layer_name = "model.layers.{}.mlp.down_proj".format(layer),
        stats_dir = "testing",
        ds_name = "wikipedia",
        to_collect = ["mom2"],
        model_name = model_name,
        sample_size = sample_size,
        precision = model_precision_str,
        batch_tokens = None,
        download = False,
        batch_size=1
    ))
    torch.save(torch.inverse(stat.mom2.mom2/stat.mom2.count),"model.layers.{}.mlp.down_proj".format(layer)+'_'+model_precision_str+'_mom2_'+'inverse.pt')

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Downloading: 100%|██████████| 15.3k/15.3k [00:00<00:00, 10.4MB/s]
Downloading: 100%|██████████| 20.3G/20.3G [08:20<00:00, 40.5MB/s] 
100%|██████████| 1000/1000 [00:54<00:00, 18.30it/s]


there were 0 problems
